In [7]:
from datasets import load_dataset
import numpy as np
import pandas as pd
from evaluate import load


In [8]:
# data = datasets.load_dataset("DFKI-SLT/cross_ner", "conll2003"
def get_data(dataset, config):
    return load_dataset(dataset, config)



In [9]:
datasets = {
    "universalner/universal_ner": ['ceb_gja', 'zh_gsd', 'zh_gsdsimp', 'zh_pud', 'hr_set', 'da_ddt', 'en_ewt', 'en_pud', 'de_pud', 'pt_bosque', 'pt_pud', 'ru_pud', 'sr_set', 'sk_snk', 'sv_pud', 'sv_talbanken', 'tl_trg', 'tl_ugnayan'],
    "DFKI-SLT/cross_ner" : ['ai', 'conll2003', 'literature', 'music', 'politics', 'science']
}

In [10]:
def label_map(data): # Accessing the label names from the 'ner_tags' feature.
    label_names = data['test'].features['ner_tags'].feature.names
    # Create mapping from label ID to label string name
    get_label = {k: v for k, v in enumerate(label_names)} 
# Create reverse mapping from label name to label ID
    get_id = {v: k for k, v in enumerate(label_names)}
    return lambda x : get_label[x], lambda label : get_id[label]



In [11]:
def recognize_named_entities(dataset):
    id2label, label2id = label_map(dataset)
    border = int(len(dataset['test']) // 2)
    # print(border)
    train_df = pd.DataFrame(dataset['test']).iloc[:border] if len(dataset) == 1 else pd.DataFrame(dataset['train'])
    test_df = pd.DataFrame(dataset['test']).iloc[border:] if len(dataset) == 1 else pd.DataFrame(dataset['test'])
    # validation_df = pd.DataFrame(dataset['validation'])
    # train_df = pd.concat([train_df, validation_df], ignore_index=True)
    named_entity_dict = {(tok, id2label(prev)): id2label(tag) for _, row in train_df.iterrows() for tok, prev, tag in zip(row['tokens'], [0] + row['ner_tags'][:-1], row['ner_tags']) if tag}
    recognized_entities = []
    for sentence in test_df['tokens']:
        last_tag = 'O'
        pred_sentence = []
        for word in sentence:
            if (word, last_tag) in named_entity_dict:
                last_tag = named_entity_dict[(word, last_tag)]
                pred_sentence.append(last_tag)
            else:
                last_tag = 'O'
                pred_sentence.append(last_tag)
        recognized_entities.append(pred_sentence)
    metric = load("seqeval")
    res = metric.compute(predictions=recognized_entities, references=[[id2label(tag) for tag in sentence]  for sentence in test_df['ner_tags']])
    return res['overall_f1'], res['overall_precision'], res['overall_recall'] 

In [12]:
for dataset, configs in datasets.items():
    print(dataset)
    for cfg in configs:
        print('\t', cfg)
        data = get_data(dataset, cfg)
        print('\t', data['test'].features['ner_tags'].feature.names)
        f1, precision, recall = recognize_named_entities(data)
        print(f"\tprecision: {precision}, recall: {recall}, f1: {f1}\n")




universalner/universal_ner
	 ceb_gja
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


/home/kirill/.cache/pypoetry/virtualenvs/bert-JeHI3B9r-py3.11/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


	precision: 1.0, recall: 0.047619047619047616, f1: 0.0909090909090909

	 zh_gsd
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.2044491525423729, recall: 0.2516297262059974, f1: 0.2255990648743425

	 zh_gsdsimp
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.052884615384615384, recall: 0.17300131061598953, f1: 0.08100644369438478

	 zh_pud
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.37320574162679426, recall: 0.12322274881516587, f1: 0.1852731591448931

	 hr_set
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.5087260034904014, recall: 0.4155381325730577, f1: 0.45743428795606117

	 da_ddt
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.20938023450586266, recall: 0.2796420581655481, f1: 0.23946360153256704

	 en_ewt
	 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
	precision: 0.1596816976127321, recall: 0.2766544117647059, f1: 0.2024

/home/kirill/.cache/pypoetry/virtualenvs/bert-JeHI3B9r-py3.11/lib/python3.11/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


	precision: 0.0, recall: 0.0, f1: 0.0

DFKI-SLT/cross_ner
	 ai
	 ['O', 'B-academicjournal', 'I-academicjournal', 'B-album', 'I-album', 'B-algorithm', 'I-algorithm', 'B-astronomicalobject', 'I-astronomicalobject', 'B-award', 'I-award', 'B-band', 'I-band', 'B-book', 'I-book', 'B-chemicalcompound', 'I-chemicalcompound', 'B-chemicalelement', 'I-chemicalelement', 'B-conference', 'I-conference', 'B-country', 'I-country', 'B-discipline', 'I-discipline', 'B-election', 'I-election', 'B-enzyme', 'I-enzyme', 'B-event', 'I-event', 'B-field', 'I-field', 'B-literarygenre', 'I-literarygenre', 'B-location', 'I-location', 'B-magazine', 'I-magazine', 'B-metrics', 'I-metrics', 'B-misc', 'I-misc', 'B-musicalartist', 'I-musicalartist', 'B-musicalinstrument', 'I-musicalinstrument', 'B-musicgenre', 'I-musicgenre', 'B-organisation', 'I-organisation', 'B-person', 'I-person', 'B-poem', 'I-poem', 'B-politicalparty', 'I-politicalparty', 'B-politician', 'I-politician', 'B-product', 'I-product', 'B-programlang', 'I